In [1]:
import pandas as pd
import numpy as np

print(" --- GERAÇÃO DE RELATÓRIO --- ")

# Carregando base de dados (nome do arquivo renomeado com dados.xlsx)
try:
    df = pd.read_excel('dados.xlsx', header=None, skiprows=1)

    # Ajuste de colunas e tipos
    df = df.iloc[:, [4, 5, 6, 7, 9, 10]]
    df.columns = ['dt_emissao', 'dt_vencimento', 'vlr_aquisicao', 'vlr_face', 'dt_pagamento', 'vlr_liquidacao']

    for col in ['dt_emissao', 'dt_vencimento', 'dt_pagamento']:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    for col in ['vlr_aquisicao', 'vlr_face', 'vlr_liquidacao']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    print(" Dados carregados com sucesso.")

except Exception as e:
    print(f" Erro crítico: {e}")

# Tarefa 1: Matriz de Adimplência
df['safra'] = df['dt_vencimento'].dt.to_period('M')
df['dias_atraso'] = (df['dt_pagamento'] - df['dt_vencimento']).dt.days

# Buckets (Regras de Negócio)
df['rec_30d'] = np.where(df['dias_atraso'] <= 30, df['vlr_liquidacao'], 0)
df['rec_60d'] = np.where(df['dias_atraso'] <= 60, df['vlr_liquidacao'], 0)
df['rec_90d'] = np.where(df['dias_atraso'] <= 90, df['vlr_liquidacao'], 0)
df['rec_180d'] = np.where(df['dias_atraso'] <= 180, df['vlr_liquidacao'], 0)
df['em_aberto'] = np.where(pd.isna(df['dt_pagamento']), df['vlr_face'], 0)

# Agrupamento
matriz = df.groupby('safra').agg({
    'vlr_face': 'sum', 'em_aberto': 'sum',
    'rec_30d': 'sum', 'rec_60d': 'sum', 'rec_90d': 'sum', 'rec_180d': 'sum'
}).reset_index()

# Calculando Percentuais
cols_rec = ['rec_30d', 'rec_60d', 'rec_90d', 'rec_180d']
cols_final = ['Liq. até 30d', 'Liq. até 60d', 'Liq. até 90d', 'Liq. até 180d']

for c_old, c_new in zip(cols_rec, cols_final):
    if matriz['vlr_face'].sum() > 0:
        matriz[c_new] = matriz[c_old] / matriz['vlr_face']
    else:
        matriz[c_new] = 0.0

matriz['% Carteira'] = matriz['vlr_face'] / matriz['vlr_face'].sum()

# Tarefa 3: Tabela de Estatísticas
colunas_destino = ['Até 30d após vcto', 'Até 60d após vcto', 'Até 90d após vcto', 'Até 180d após vcto']
linha_media_simples = []
linha_media_ponderada = []
linha_desvio = []
linha_inadimplencia = []

for col in cols_final:
    media = matriz[col].mean()
    media_pond = (matriz[col] * matriz['vlr_face']).sum() / matriz['vlr_face'].sum()
    desvio = matriz[col].std()
    inadimplencia = 1 - media_pond

    linha_media_simples.append(media)
    linha_media_ponderada.append(media_pond)
    linha_desvio.append(desvio)
    linha_inadimplencia.append(inadimplencia)

dados_tabela_3 = {
    'Cálculo': ['Média Adimplência', 'Média Ponderada - Adimplência', 'Desvio Padrão', 'Inadimplência']
}
for i, nome_coluna in enumerate(colunas_destino):
    dados_tabela_3[nome_coluna] = [
        linha_media_simples[i], linha_media_ponderada[i],
        linha_desvio[i], linha_inadimplencia[i]
    ]
tabela_stats = pd.DataFrame(dados_tabela_3)

# 4. PREPARAÇÃO FINAL E EXPORTAÇÃO
tabela_vintage = matriz[['safra', 'vlr_face', '% Carteira', 'em_aberto'] + cols_final].copy()
tabela_vintage.rename(columns={'safra': 'Vencimento', 'vlr_face': 'Valor Total', 'em_aberto': 'Em Aberto'}, inplace=True)

# Verificação de Erros
qtd_erros = 0
erros_lista = []
if 'dt_pagamento' in df.columns:
    qtd_erros = len(df[df['dt_pagamento'] < df['dt_emissao']])
    erros_lista = [['Pagto antes Emissão', qtd_erros]]

# Salvando Excel (Dados Puros)
nome_arquivo = "Relatorio_Analise_Carteira_Final.xlsx"
with pd.ExcelWriter(nome_arquivo) as writer:
    tabela_vintage.to_excel(writer, sheet_name='Matriz adimplencia', index=False)
    tabela_stats.to_excel(writer, sheet_name='Estatisticas', index=False)
    pd.DataFrame(erros_lista, columns=['Tipo', 'Qtd']).to_excel(writer, sheet_name='Inconsistencia', index=False)

print(f"\n Arquivo Excel '{nome_arquivo}' gerado!")
print(f" Inconsistências encontradas: {qtd_erros}")

# 5. VISUALIZAÇÃO NA TELA (Formatada para Leitura)
print("\n--- MATRIZ DE ADIMPLÊNCIA ---")
# formatação visual
display(tabela_vintage.style.format({
    'Valor Total': 'R$ {:,.2f}',       # Ex: R$ 1.000.000,00
    '% Carteira': '{:.2%}',            # Ex: 12.34%
    'Em Aberto': 'R$ {:,.2f}',         # Ex: R$ 0.00
    'Liq. até 30d': '{:.2%}',
    'Liq. até 60d': '{:.2%}',
    'Liq. até 90d': '{:.2%}',
    'Liq. até 180d': '{:.2%}'
}))

print("\n--- RESUMO DAS ESTATÍSTICAS ---")
display(tabela_stats.style.format({c: '{:.2%}' for c in colunas_destino}))

 --- GERAÇÃO DE RELATÓRIO --- 
 Dados carregados com sucesso.

 Arquivo Excel 'Relatorio_Analise_Carteira_Final.xlsx' gerado!
 Inconsistências encontradas: 0

--- MATRIZ DE ADIMPLÊNCIA ---


,Vencimento,Valor Total,% Carteira,Em Aberto,Liq. até 30d,Liq. até 60d,Liq. até 90d,Liq. até 180d
0,2020-03,"R$ 11,956,750.05",1.31%,R$ 0.00,100.00%,100.00%,100.00%,100.00%
1,2020-04,"R$ 5,000,000.00",0.55%,R$ 0.00,50.00%,100.00%,100.00%,100.00%
2,2020-07,"R$ 480,949.48",0.05%,R$ 0.00,0.00%,0.00%,0.00%,0.00%
3,2020-08,"R$ 14,880,422.30",1.63%,"R$ 3,940,827.28",33.29%,33.29%,33.29%,42.25%
4,2021-02,"R$ 4,741,125.50",0.52%,R$ 0.00,0.00%,100.00%,100.00%,100.00%
5,2021-03,"R$ 46,644,373.02",5.11%,"R$ 1,300,847.75",60.03%,69.96%,77.17%,82.96%
6,2021-04,"R$ 37,084,182.72",4.07%,"R$ 2,963,482.56",73.29%,85.22%,89.31%,90.11%
7,2021-05,"R$ 42,123,691.53",4.62%,"R$ 362,540.72",85.70%,86.61%,87.21%,92.30%
8,2021-06,"R$ 14,645,107.16",1.61%,"R$ 152,824.54",97.25%,97.25%,97.25%,97.86%
9,2021-07,"R$ 9,949,049.39",1.09%,"R$ 1,442,145.33",66.74%,67.89%,73.44%,74.59%



--- RESUMO DAS ESTATÍSTICAS ---


,Cálculo,Até 30d após vcto,Até 60d após vcto,Até 90d após vcto,Até 180d após vcto
0,Média Adimplência,70.90%,80.58%,84.46%,87.31%
1,Média Ponderada - Adimplência,77.01%,84.46%,88.99%,90.61%
2,Desvio Padrão,31.09%,27.53%,27.10%,22.08%
3,Inadimplência,22.99%,15.54%,11.01%,9.39%
